In [ ]:
!pip install keras-tuner

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, LSTM, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras_tuner.tuners import BayesianOptimization
from keras.optimizer_v2 import adam as adam_v2
import os
import math
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df_org = pd.read_csv("/content/drive/MyDrive/DC2/all_variables.csv")
df_org

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Year,Month,LSOA_code,LSOA_name,Type_I_crime_amount,Total_month,Type_II_crime_amount
0,2011,1,E01000001,City of London 001A,2,1,8
1,2011,2,E01000001,City of London 001A,1,2,8
2,2011,3,E01000001,City of London 001A,6,3,4
3,2011,4,E01000001,City of London 001A,1,4,4
4,2011,6,E01000001,City of London 001A,1,6,15
...,...,...,...,...,...,...,...
3286689,2021,6,W01001958,Swansea 025H,6,126,16
3286690,2021,7,W01001958,Swansea 025H,4,127,11
3286691,2021,8,W01001958,Swansea 025H,9,128,10
3286692,2021,9,W01001958,Swansea 025H,4,129,16


In [ ]:
# E01000005 for Urban
# E01030759 for Town and Fringe
# E01020795 for Village Hamlet & Isolated Dwellings

In [ ]:
# Extract the data from a given LSOA code
df_need = df_org[df_org['LSOA_code'] == 'E01000005'].reset_index(drop = True)
df_need = df_need[['Total_month', 'Type_I_crime_amount', 'Type_II_crime_amount']]
df_need = df_need[df_need['Total_month'] <= 108]
df_need.drop(columns = 'Total_month', inplace = True)
df_need

,Type_I_crime_amount,Type_II_crime_amount
0,7,21
1,10,30
2,6,31
3,5,26
4,2,37
...,...,...
103,12,30
104,8,34
105,12,34
106,12,40


In [ ]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences) - 1:
          break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i: end_ix], sequences[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# Hyperparameter searching if necessary

def build_model(hp):
    # Build the LSTM model with 2 LSTM layers and 1 Dense layer
    input = Input(shape = (12, 2))
    l1 = LSTM(units = hp.Int('units_1', min_value = 50, max_value = 300, step = 50), return_sequences = True, dropout = 0.2)(input)
    b1 = BatchNormalization()(l1)
    l2 = LSTM(units = hp.Int('units_2', min_value = 50, max_value = 300, step = 50), return_sequences = False, dropout = 0.2)(b1)
    b2 = BatchNormalization()(l2)
    output = Dense(2)(b2)
    model = Model(input, output)

    # The optimizer is Adam
    opt = adam_v2.Adam(learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4, 2e-2, 2e-3, 2e-4]))
    model.compile(loss = 'mse', optimizer = opt, metrics = ['mse'])

    return model

def find_best_model():
    # Example of splitting the data when time step is 12 months: 
    # Training data has data from 1st month to 24th month
    # Val data has data from 13th month to 36th month
    # Test data predicts new data using data from 25th month to 36th month
    train_data = df_need.to_numpy()[: 96]
    val_data = df_need.to_numpy()[84: 108]
    # Split x and y arrays using sliding window approach
    x_train, y_train = split_sequences(train_data, 12)
    x_val, y_val = split_sequences(val_data, 12)
    bayesian_opt_tuner = BayesianOptimization(build_model, objective = 'mse',
        max_trials = 216,
        executions_per_trial = 1,
        directory = '/content/drive/MyDrive/DC2/tune/Urban/',
        project_name = 'kerastuner_bayesian_poc',
        overwrite = True)

    bayesian_opt_tuner.search(x_train, y_train, epochs = 500, validation_data = (x_val, y_val))

    bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models = 1)
    model = bayes_opt_model_best_model[0]
    return model

#model = find_best_model()
#model.summary()

In [ ]:
def train(df, train_nr, prediction_nr, comparison, model_exist, n_steps=12):
    with tf.device(device_name):
        # Example of splitting the data when time step is 12 months: 
        # Training data has data from 1st month to 24th month
        # Val data has data from 13th month to 36th month
        # Test data predicts new data using data from 25th month to 36th month
        train_data = df.to_numpy()[: train_nr - n_steps]
        val_data = df.to_numpy()[train_nr - n_steps * 2: train_nr]
        # Split x and y arrays using sliding window approach
        x_train, y_train = split_sequences(train_data, n_steps)
        x_val, y_val = split_sequences(val_data, n_steps)

        if model_exist == "Nil":
            # Build the LSTM model with 2 LSTM layers and 1 Dense layer
            input = Input(shape = (n_steps, 2))
            l1 = LSTM(50, return_sequences = True, dropout = 0.2)(input)
            b1 = BatchNormalization()(l1)
            l2 = LSTM(50, return_sequences = False, dropout = 0.2)(b1)
            b2 = BatchNormalization()(l2)
            output = Dense(2)(b2)
            model = Model(input, output)

            # The optimizer is Adam
            opt = adam_v2.Adam(learning_rate = 0.005, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08)

            # The loss used in model training is mean_squared_error because it is crime number prediction
            model.compile(loss = 'mse', optimizer = opt)

        else:
            model = model_exist
        
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 200)
        checkpoint_filepath = '/content/drive/MyDrive/DC2/model/Urban/' + str(train_nr) + '/weights.{epoch:02d}.h5'
        model_checkpoint_callback = ModelCheckpoint(filepath = checkpoint_filepath, monitor = 'val_loss', mode = 'min', save_best_only = True)
        lr_reducer = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 50, verbose = 0, mode = 'min', min_delta = 0.0001, cooldown = 0, min_lr = 0)

        # Fit the model with 1000 epoches and batch size 64
        # Validation data is used here for evaluation during the training process
        model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 1000, batch_size = 32, callbacks = [model_checkpoint_callback, lr_reducer, early_stopping])

        # Look for all file directories in the folder
        directory = "/content/drive/MyDrive/DC2/model/Urban/" + str(train_nr) + '/'
        all_file_lst = []
        for root, subdirectories, files in os.walk(directory):
            for file in files:
                all_file_lst.append(os.path.join(root, file))
        # Load the best model trained
        model = load_model(all_file_lst[-1])

        # Make predictions from testing data
        # Pred x is a list of predicted data used
        # Pred y is a list of predicted result
        # Get the last 12 results from overall training data for prediction data in pred x
        pred_x = np.array(list(val_data[-(n_steps): ]))
        pred_x = pred_x.reshape((1, pred_x.shape[0], pred_x.shape[1]))
        # Initialize pred y as empty array
        pred_y = np.empty((1, 1, 2), int)
        for i in range(prediction_nr):
            # Always round off the latest prediction to integer because number of crime can only be integer
            prediction = np.round(model.predict(pred_x))
            # Update pred y 
            pred_y = np.append(pred_y, [prediction], axis = 1)
            # Update pred x by feeding the latest prediction result as latest feature
            pred_x = np.append(pred_x, [prediction], axis = 1)
            # Update pred x by removing the 1st row because of the timestep size
            pred_x = pred_x[: , 1: , : ]
        # Remove the 1st row of pred y because when initializing array, random numbers are assigned which are not actual results
        pred_y = pred_y[: , 1: , : ]
        
        df_predict = pd.DataFrame(columns = ['Type_I_crime_amount', 'Type_II_crime_amount'])
        df_predict['Type_I_crime_amount'] = np.round(pred_y[: , : , 0])[0]
        df_predict['Type_II_crime_amount'] = np.round(pred_y[: , : , 1])[0]

        if comparison == False:
            return df_predict
        else:
            # Compute RMSE, MAE and R2 by comparing predicted result from actual crime number
            y_true = df[train_nr: (train_nr + prediction_nr)]
            rmse_1 = math.sqrt(mean_squared_error(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist()))
            rmse_2 = math.sqrt(mean_squared_error(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist()))
            mae_1 = mean_absolute_error(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            mae_2 = mean_absolute_error(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            medae_1 = median_absolute_error(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            medae_2 = median_absolute_error(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            r2_1 = r2_score(y_true['Type_I_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            r2_2 = r2_score(y_true['Type_II_crime_amount'].tolist(), df_predict['Type_I_crime_amount'].tolist())
            return df_predict, rmse_1, rmse_2, mae_1, mae_2, medae_1, medae_2, r2_1, r2_2

In [ ]:
%%time
df_compare_predict = pd.DataFrame(columns = ['Type_I_crime_amount', 'Type_II_crime_amount'])
rmse_lst_1 = []
rmse_lst_2 = []
mae_lst_1 = []
mae_lst_2 = []
medae_lst_1 = []
medae_lst_2 = []
r2_lst_1 = []
r2_lst_2 = []

# Month Forward-Chaining Cross Validation Approach is used below
for i in tqdm(range(6)):
    df_small, rmse_1_small, rmse_2_small, mae_1_small, mae_2_small, medae_1_small, medae_2_small, r2_1_small, r2_2_small = train(df_need, 36 + 12 * i, 12, True, "Nil")
    df_compare_predict = pd.concat([df_compare_predict, df_small])
    rmse_lst_1.append(rmse_1_small)
    rmse_lst_2.append(rmse_2_small)
    mae_lst_1.append(mae_1_small)
    mae_lst_2.append(mae_2_small)
    medae_lst_1.append(medae_1_small)
    medae_lst_2.append(medae_2_small)    
    r2_lst_1.append(r2_1_small)
    r2_lst_2.append(r2_2_small)

df_compare_predict

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/1000
1/1 [==============================] - 12s 12s/step - loss: 783.1092 - val_loss: 563.1605 - lr: 0.0050
Epoch 2/1000
1/1 [==============================] - 0s 281ms/step - loss: 775.1265 - val_loss: 560.5746 - lr: 0.0050
Epoch 3/1000
1/1 [==============================] - 0s 275ms/step - loss: 779.1465 - val_loss: 555.9689 - lr: 0.0050
Epoch 4/1000
1/1 [==============================] - 0s 282ms/step - loss: 772.5908 - val_loss: 552.1393 - lr: 0.0050
Epoch 5/1000
1/1 [==============================] - 0s 237ms/step - loss: 770.3919 - val_loss: 551.5323 - lr: 0.0050
Epoch 6/1000
1/1 [==============================] - 0s 288ms/step - loss: 769.4590 - val_loss: 549.7505 - lr: 0.0050
Epoch 7/1000
1/1 [==============================] - 0s 259ms/step - loss: 765.5942 - val_loss: 548.4084 - lr: 0.0050
Epoch 8/1000
1/1 [==============================] - 0s 325ms/step - loss: 772.0591 - val_loss: 544.8023 - lr: 0.0050
Epoch 9/1000
1/1 [==============================] - 0s 228ms/step

 17%|█▋        | 1/6 [00:51<04:19, 51.93s/it]

Epoch 1/1000
1/1 [==============================] - 6s 6s/step - loss: 673.5446 - val_loss: 495.9292 - lr: 0.0050
Epoch 2/1000
1/1 [==============================] - 0s 116ms/step - loss: 667.4454 - val_loss: 494.1566 - lr: 0.0050
Epoch 3/1000
1/1 [==============================] - 0s 127ms/step - loss: 659.2734 - val_loss: 490.0960 - lr: 0.0050
Epoch 4/1000
1/1 [==============================] - 0s 145ms/step - loss: 659.6562 - val_loss: 486.0714 - lr: 0.0050
Epoch 5/1000
1/1 [==============================] - 0s 116ms/step - loss: 661.1597 - val_loss: 484.1804 - lr: 0.0050
Epoch 6/1000
1/1 [==============================] - 0s 44ms/step - loss: 664.5955 - val_loss: 484.3848 - lr: 0.0050
Epoch 7/1000
1/1 [==============================] - 0s 49ms/step - loss: 651.8609 - val_loss: 486.4997 - lr: 0.0050
Epoch 8/1000
1/1 [==============================] - 0s 45ms/step - loss: 648.3925 - val_loss: 486.9741 - lr: 0.0050
Epoch 9/1000
1/1 [==============================] - 0s 47ms/step - los

 33%|███▎      | 2/6 [01:40<03:20, 50.20s/it]

...

Epoch 1/1000
2/2 [==============================] - 5s 1s/step - loss: 546.2738 - val_loss: 577.9205 - lr: 0.0050
Epoch 2/1000
2/2 [==============================] - 0s 39ms/step - loss: 538.2002 - val_loss: 586.6618 - lr: 0.0050
Epoch 3/1000
2/2 [==============================] - 0s 113ms/step - loss: 541.2114 - val_loss: 575.2105 - lr: 0.0050
Epoch 4/1000
2/2 [==============================] - 0s 120ms/step - loss: 535.9244 - val_loss: 565.2703 - lr: 0.0050
Epoch 5/1000
2/2 [==============================] - 0s 42ms/step - loss: 528.5172 - val_loss: 568.0727 - lr: 0.0050
Epoch 6/1000
2/2 [==============================] - 0s 43ms/step - loss: 524.1827 - val_loss: 566.9638 - lr: 0.0050
Epoch 7/1000
2/2 [==============================] - 0s 113ms/step - loss: 520.0506 - val_loss: 561.9680 - lr: 0.0050
Epoch 8/1000
2/2 [==============================] - 0s 111ms/step - loss: 516.9675 - val_loss: 553.4391 - lr: 0.0050
Epoch 9/1000
2/2 [==============================] - 0s 115ms/step - lo

 83%|████████▎ | 5/6 [03:35<00:42, 42.39s/it]

Epoch 1/1000
3/3 [==============================] - 6s 680ms/step - loss: 554.5953 - val_loss: 742.0412 - lr: 0.0050
Epoch 2/1000
3/3 [==============================] - 0s 61ms/step - loss: 546.2175 - val_loss: 728.9532 - lr: 0.0050
Epoch 3/1000
3/3 [==============================] - 0s 69ms/step - loss: 538.0629 - val_loss: 714.6638 - lr: 0.0050
Epoch 4/1000
3/3 [==============================] - 0s 62ms/step - loss: 537.7101 - val_loss: 711.1800 - lr: 0.0050
Epoch 5/1000
3/3 [==============================] - 0s 30ms/step - loss: 531.8442 - val_loss: 715.4899 - lr: 0.0050
Epoch 6/1000
3/3 [==============================] - 0s 27ms/step - loss: 524.4498 - val_loss: 712.2538 - lr: 0.0050
Epoch 7/1000
3/3 [==============================] - 0s 62ms/step - loss: 516.0999 - val_loss: 706.5089 - lr: 0.0050
Epoch 8/1000
3/3 [==============================] - 0s 72ms/step - loss: 509.9872 - val_loss: 691.7772 - lr: 0.0050
Epoch 9/1000
3/3 [==============================] - 0s 63ms/step - loss

100%|██████████| 6/6 [04:16<00:00, 42.70s/it]

CPU times: user 3min 2s, sys: 14.3 s, total: 3min 16s
Wall time: 4min 16s


In [ ]:
rmse_lst_1, rmse_lst_2

([4.320493798938574,
  3.851406669430448,
  3.2015621187164243,
  3.2015621187164243,
  5.809475019311125,
  5.212165257037297],
 [23.512408071767837,
  18.79716290649558,
  21.550328690455437,
  24.07972868064201,
  26.78619047195775,
  33.4165627596057])

In [ ]:
mae_lst_1, mae_lst_2

([3.8333333333333335,
  2.8333333333333335,
  2.4166666666666665,
  2.5833333333333335,
  4.583333333333333,
  4.333333333333333],
 [22.833333333333332,
  18.333333333333332,
  20.25,
  22.0,
  25.833333333333332,
  31.166666666666668])

In [ ]:
medae_lst_1, medae_lst_2

([3.5, 2.0, 2.0, 2.0, 3.0, 4.0], [23.5, 18.0, 20.0, 21.5, 23.5, 26.0])

In [ ]:
r2_lst_1, r2_lst_2

([-2.7178423236514524,
  -0.2654028436018956,
  -0.08929889298892979,
  -0.06034482758620685,
  -0.6718266253869969,
  -1.7414155571128243],
 [-16.60070749502543,
  -19.483091787439612,
  -7.5442698351858954,
  -5.066700574002762,
  -12.783351120597656,
  -6.424165473936932])

In [ ]:
rmse_1 = sum(rmse_lst_1) / len(rmse_lst_1)
rmse_2 = sum(rmse_lst_2) / len(rmse_lst_2)
mae_1 = sum(mae_lst_1) / len(mae_lst_1)
mae_2 = sum(mae_lst_2) / len(mae_lst_2)
medae_1 = sum(medae_lst_1) / len(medae_lst_1)
medae_2 = sum(medae_lst_2) / len(medae_lst_2)
r2_1 = sum(r2_lst_1) / len(r2_lst_1)
r2_2 = sum(r2_lst_2) / len(r2_lst_2)

print("Type I crime amount: \n RMSE: " + str(rmse_1) + "\n MAE: " + str(mae_1) + "\n MEDAE: " + str(medae_1) + "\n R2: " + str(r2_1) + 
      "\n \n Type II crime amount: \n RMSE: " + str(rmse_2) + "\n MAE: " + str(mae_2) + "\n MEDAE: " + str(medae_1) + "\n R2: " + str(r2_2))

Type I crime amount: 
 RMSE: 4.266110830358382
 MAE: 3.4305555555555554
 MEDAE: 2.75
 R2: -0.9243551783880509
 
 Type II crime amount: 
 RMSE: 24.69039693015405
 MAE: 23.402777777777775
 MEDAE: 2.75
 R2: -11.317047714364714


In [ ]:
df_real_predict = train(df_need, 108, 24, False, "Nil")
df_real_predict

Epoch 1/1000
3/3 [==============================] - 6s 658ms/step - loss: 581.3675 - val_loss: 944.2668 - lr: 0.0050
Epoch 2/1000
3/3 [==============================] - 0s 59ms/step - loss: 575.4476 - val_loss: 935.4475 - lr: 0.0050
Epoch 3/1000
3/3 [==============================] - 0s 63ms/step - loss: 567.3640 - val_loss: 928.8610 - lr: 0.0050
Epoch 4/1000
3/3 [==============================] - 0s 59ms/step - loss: 561.4554 - val_loss: 922.5911 - lr: 0.0050
Epoch 5/1000
3/3 [==============================] - 0s 61ms/step - loss: 556.1551 - val_loss: 916.9710 - lr: 0.0050
Epoch 6/1000
3/3 [==============================] - 0s 60ms/step - loss: 549.6816 - val_loss: 910.9499 - lr: 0.0050
Epoch 7/1000
3/3 [==============================] - 0s 59ms/step - loss: 543.0044 - val_loss: 892.5588 - lr: 0.0050
Epoch 8/1000
3/3 [==============================] - 0s 67ms/step - loss: 535.6238 - val_loss: 879.2856 - lr: 0.0050
Epoch 9/1000
3/3 [==============================] - 0s 26ms/step - loss

,Type_I_crime_amount,Type_II_crime_amount
0,11.0,38.0
1,10.0,36.0
2,9.0,34.0
3,7.0,29.0
4,8.0,24.0
5,8.0,24.0
6,12.0,24.0
7,13.0,24.0
8,14.0,25.0
9,13.0,26.0


In [ ]:
df_compare_predict.to_csv("/content/drive/MyDrive/DC2/urban_old_prediction.csv", index = False)
df_real_predict.to_csv("/content/drive/MyDrive/DC2/urban_new_prediction.csv", index = False)